# Evaluate our ChromaDB with its Embeddings model

In [ ]:
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import chromadb


db_path="./vektor_DB"
client = chromadb.PersistentClient(path=db_path)

embedding_model = SentenceTransformer("sentence-transformers/gtr-t5-large")

collection = client.get_collection("meinungen")

In [ ]:
def query_collection(query_text, n_results=6):
    """Queries the ChromaDB collection and returns relevant results."""

    query_embedding = embedding_model.encode(query_text)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results,
        include=["documents", "metadatas", "distances"]
    )
    return results

query = "Privacy"
results = query_collection(query)

print(f"Query: {query}")
for i in range(len(results["documents"][0])):
    print(f"\nResult {i+1}:")
    print(f"  Document: {results['documents'][0][i]}")
    print(f"  Party: {results['metadatas'][0][i]['party']}")
    print(f"  Theme: {results['metadatas'][0][i]['theme']}")
    print(f"  Chunk Index: {results['metadatas'][0][i]['chunk_index']}")
    print(f"  Distance: {results['distances'][0][i]}")

Query: Privacy

Result 1:
  Document: Wir Freie Demokraten wollen den Schutz der Menschenrech- te im digitalen Zeitalter stärken. Hierfür wollen wir die Inter- netfreiheit und digitalen Menschenrechte zu neuen außen- politischen Schwerpunkten machen. Denn die Digitalisierung birgt sowohl neue Chancen für den Schutz der Menschen- rechte als auch neue Herausforderungen. Beim Ausbau des Menschenrechtsschutzes im digitalen Zeitalter muss Deutsch- land im Zuge einer effektiven Cybersicherheitsstrategie in Europa und international eine Führungsrolle einnehmen – insbesondere beim Recht auf Privatsphäre, Anonymität im Internet und Verschlüsselung sowie beim Schutz personen- bezogener Daten und vor Massenüberwachung. Zum Schutz der Privatsphäre gehört auch, dass zur Straf- und Zivilrechts- verfolgung von Persönlichkeitsrechtsverletzungen Anonymität aufgehoben werden kann.
  Party: FDP
  Theme: Menschenrechte im digitalen Zeitalter schützen
  Chunk Index: 0
  Distance: 0.6077107589997595

Result

## Read the data and handle long text

In [ ]:
import pandas as pd
csv_file = "Daten.csv"
data = pd.read_csv(csv_file)

#Funktion
def chunk_text(text, max_tokens=512):
    tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/gtr-t5-large")
    words = text.split()
    chunks = []
    while len(words) > 0:
        chunk = " ".join(words[:max_tokens])
        chunks.append(chunk)
        words = words[max_tokens:]
    return chunks

## Eval the vektor_DB with cross validation and KNN classifier

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

#Extract features (embeddings) und labels (partei)
embeddings = []
labels = []

for index, row in data.iterrows():
    party = row["Partei"]
    long_text = row["Meinung"]

    if isinstance(long_text, str):
        chunks = chunk_text(long_text)
        for chunk in chunks:
            embedding = embedding_model.encode(chunk)
            embeddings.append(embedding)
            labels.append(party)


embeddings = np.array(embeddings)
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

knn_classifier = KNeighborsClassifier(n_neighbors=5)

#Cross-validation um modell zu evaluieren
cv_scores = cross_val_score(knn_classifier, X_train, y_train, cv=5, scoring='accuracy')
print(f"Cross-Validation Accuracy: Mean = {cv_scores.mean()}, Std = {cv_scores.std()}")

knn_classifier.fit(X_train, y_train)

y_pred = knn_classifier.predict(X_test)

#evualte + accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the KNN classifier: {accuracy}")


Cross-Validation Accuracy: Mean = 0.3821666053245001, Std = 0.02178320723925732
Accuracy of the KNN classifier: 0.3585434173669468
